<img src="./images/DLI_Header.png" style="width: 400px;">


# 6.0 Mixture of Experts (MoE)

이번 노트북에서는 Mixture of Experts 모델 트레이닝에 대해 알아봅니다.

## 목표 

이번 노트북의 목표는 다음과 같습니다 :
* 심플 컨볼루션 네트워크에서 선형 전문가(expert) 신경망들을 통합하는 방법
* 분류 문제를 위해 새로운 Mixture of Experts CNN을 트레이닝하는 방법


### 기존 실행 중이거나 보류 상태의 Job을 취소하고 이번 노트북을 진행합니다. 

실습 진행에 앞서  SLURM 대기열에서 아직 실행 중이거나 대기 중인 작업이 없는지 확인하십시오. 다음 셀을 실행하여 SLURM 작업 대기열을 확인합니다. :



In [3]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하고 `scancel` 명령을 사용하여 모든 사용자의 작업을 취소합니다.

In [4]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 6.1 Mixture of Experts 개요

Mixture of Experts(MoE)은 일부 레이어가 상황에 따라 활성화되거나 활성화되지 않는 작은 그룹으로 분할되는 신경망입니다.
이 구조는 네트워크가 더 넓은 범위의 동작을 학습할 수 있게 합니다. 또 다른 장점은 한 번에 소수의 Expert 신경망들만 활동하기 때문에 MoE 모델이 더 적은 연산을 필요로 한다는 것입니다.

<img src="images/MOE.png" width="450" />

최근 논문에서는 [Switch Transformer](https://arxiv.org/pdf/2101.03961.pdf)와 같은 MoE 구조를 따르는 여러 모델이 개발되었습니다.
 

# 6.2 베이스라인 CNN을 위한 MoE 작성 방법

CNN cifar-10 분류기 모델로 돌아가서 MoE 레이어 1개를 추가하도록 수정해 보겠습니다. CNN 컨볼루션 레이어들은 특징들을 추출하며, 나중에 완전히 연결된 레이어는 CIFAR-10 분류 문제에 특화되어 있습니다. 
네트워크 정의에 전문가 계층을 추가하려면 다음과 같이`deepspeed.moe.layer.MoE`를 사용하십시오 (정방향 패스를 수정합니다).:

```
deepspeed.moe.layer.MoE( hidden_size=<Hidden dimension of the model>, 
                         expert=<Torch module that defines the expert>, 
                         num_experts=<Desired number of expert>, 
                         ep_size=<Desired expert-parallel world size>,
                         ...
                         )
                         
```

[DeepSpeed 설명서](https://deepspeed.readthedocs.io/en/latest/moe.html)에서 DeepSpeed의 Mixture of Experts에 대해서 더 알아보세요.

초기 레이어에서 추출된 특징을 평가하기 위해 최신의 완전 연결 레이어 `fc3` 를 MoE 레이어로 변환해 보겠습니다. 최종 분류자인 `fc4`를 추가하겠습니다.

우리는 이미 [cifar10_deepspeed_MOE.py](./code/moe/cifar10_deepspeed_MOE.py)스크립트를 준비했습니다. 4개의 GPU에 분할된 8개의 전문가 신경망을 사용하여 실행해 보겠습니다. 다시 말하면, 각 GPU는 2개의 전문가 신경망을 처리할 예정입니다.


In [ ]:
! deepspeed --num_gpus=4 /dli/code/moe/cifar10_deepspeed_MOE.py  \
    --deepspeed \
    --deepspeed_config /dli/code/moe/ds_config.json \
    --moe \
    --ep-world-size 4 \
    --num-experts-per-layer 8 \
    --top-k 1 \
    --noisy-gate-policy 'RSample' \
    --moe-param-group \
    --profile-execution=True \
    --profile-name='zero0_MOE'

[2025-03-28 05:17:18,599] [WARNING] [runner.py:159:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2025-03-28 05:17:18,765] [INFO] [runner.py:457:main] cmd = /opt/conda/bin/python3.8 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMSwgMiwgM119 --master_addr=127.0.0.1 --master_port=29500 /dli/code/moe/cifar10_deepspeed_MOE.py --deepspeed --deepspeed_config /dli/code/moe/ds_config.json --moe --ep-world-size 4 --num-experts-per-layer 8 --top-k 1 --noisy-gate-policy RSample --moe-param-group --profile-execution=True --profile-name=zero0_MOE
[2025-03-28 05:17:20,309] [INFO] [launch.py:96:main] 0 NCCL_P2P_DISABLE=1
[2025-03-28 05:17:20,309] [INFO] [launch.py:96:main] 0 NCCL_VERSION=2.11.4
[2025-03-28 05:17:20,309] [INFO] [launch.py:103:main] WORLD INFO DICT: {'localhost': [0, 1, 2, 3]}
[2025-03-28 05:17:20,310] [INFO] [launch.py:109:main] nnodes=1, num_local_procs=4, node_rank=0
[2025-03-28 05:17:20,310] [INFO] [launch.py:1

<img src="images/deepspeed_MOE.png" width="950" />

---
<h2 style="color:green;">축하합니다!</h2>

다음 실습은 대형 신경망을 배포하는 데 초점을 맞출 것입니다.

다음으로 넘어가기 전에 대기열에서 실행 중이거나 대기 중인 작업이 없는지 확인해야 합니다.

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


아직 실행 중이거나 보류 중인 작업이 있는 경우 다음 셀을 실행하고 `scancel` 명령을 사용하여 모든 어드민 사용자의 작업을 취소합니다.

In [2]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
